In [1]:
from google.colab import drive
drive.mount('/content/drive2',force_remount=True)

Mounted at /content/drive2


In [2]:
# install stop_words if not installed
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32919 sha256=713f155008d2e3a395aa92123bde0ce7c5682d1f2906fcceab9b3a71c06b014f
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [3]:
import pandas as pd
import numpy as np
import nltk
import codecs
from sklearn import preprocessing
from stop_words import get_stop_words # could be deleted
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
import sklearn.model_selection
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
import re
import time
import scipy
import tensorflow as tf
from gensim.models import KeyedVectors
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [5]:
global all_stop_words
all_stop_words = get_stop_words('turkish')
additional_stopwords = ['acaba','ama','aslında','az','bazı','belki','biri','birkaç','birşey','biz','bu','çok','çünkü','da','daha','de','defa','diğer','eğer','en','gibi','hem','hepsi','her','hiç','için','ile','ise','kez','ki','kim','mi','mü','nasıl','ne','neden','nerde','nerede','nereye','niçin','niye','o','sanki','şey','şu','siz','tüm','ve','veya','ya','yani','nin','ın','in','nın','nde','den','dan','nda']
all_stop_words += additional_stopwords

In [111]:
train_data = pd.read_csv("/content/drive2/MyDrive/NLP/Dataset/train.csv")
test_data = pd.read_csv("/content/drive2/MyDrive/NLP/Dataset/test.csv")

In [112]:
def preprocessText(text):
  text = text.lower()
  word_list = text.split()
  for word in word_list:
    if word in all_stop_words:
      word_list.remove(word)
  text = " ".join(word_list)
  return text

In [113]:
train_data['text'] = train_data['text'].apply(lambda x: preprocessText(x))
test_data['text'] = test_data['text'].apply(lambda x: preprocessText(x))

In [114]:
train_Data, validation_Data = sklearn.model_selection.train_test_split(train_data, train_size = 0.8, random_state=0)

In [115]:
types = train_data.label.unique()
label_dict={}
for i,text_label in enumerate(types):
    label_dict[text_label]=i
labels=train_data.label.apply(lambda x: label_dict[x])

In [116]:
types = test_data.label.unique()
label_dict={}
for i,text_label in enumerate(types):
    label_dict[text_label]=i
test_labels=test_data.label.apply(lambda x:label_dict[x])

In [117]:
# another approach (working one)
texts=train_Data.text
vocab_size=1000000 # guarantee to consider all the vocabulaires
tokenizer = Tokenizer(num_words=vocab_size,filters='!"$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
sequences_valid=tokenizer.texts_to_sequences(validation_Data.text)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 166525 unique tokens.


In [118]:
#download it if required
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [119]:
# calculate the maximum length sequences in the train,validation and test data (these operations may take a few seconds)
max_length_train =- 1
for doc in train_Data['text']:
    tokens=nltk.word_tokenize(doc)
    if (max_length_train < len(tokens)):
        max_length_train = len(tokens)

max_length_validation =- 1
for doc in validation_Data['text']:
    tokens=nltk.word_tokenize(doc)
    if (max_length_validation < len(tokens)):
        max_length_validation = len(tokens)

max_length_test =- 1
for doc in test_data['text']:
    tokens=nltk.word_tokenize(doc)
    if (max_length_test < len(tokens)):
        max_length_test = len(tokens)

In [120]:
# make sure to have all the data sequences have the same length
X_train  = pad_sequences(sequences_train,maxlen = max(max_length_train,max_length_validation))
X_val    = pad_sequences(sequences_valid,maxlen = max(max_length_train,max_length_validation))
y_train  = train_Data['label']
y_val    = validation_Data['label']
label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.fit_transform(y_val)
#y_train = to_categorical(np.asarray(labels[train_Data.index]))
#y_val = to_categorical(np.asarray(labels[validation_Data.index]))
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (6400, 7513) (1600, 7513)
Shape of label train and validation tensor: (6400,) (1600,)


In [121]:
word_index = tokenizer.word_index

In [105]:
# load pretrained model
word_vectors = KeyedVectors.load_word2vec_format('/content/drive2/MyDrive/NLP/trmodel', binary=True)

In [122]:
# load pretrained model
word_vectors = KeyedVectors.load_word2vec_format('/content/drive2/MyDrive/NLP/my_model.bin', binary=True) # with 5k dataset

In [90]:
#my pretrained model
word_vectors = KeyedVectors.load_word2vec_format('/content/drive2/MyDrive/NLP/my_model_100k.bin', binary=True) # with 100k dataset

In [61]:
# without pretrained model
embedding_dim=400
vocabulary_size=min(len(word_index)+1,vocab_size)

In [123]:
embedding_dim    = 100 # it was 400 for the large pretrained model
vocabulary_size  = min(len(word_index)+1,vocab_size)
embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
for word, i in word_index.items():
    try:
      embedding_vector    = word_vectors[word]
      embedding_matrix[i] = embedding_vector
    except KeyError:
      #print("key error")
      embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)

In [41]:
# basic cnn architecture without pretrained embedding (1conv1d layer)
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,trainable = True,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [77]:
# basic cnn architecture without pretrained embedding (1conv1d layer) static CNN
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,trainable = False,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [23]:
# basic cnn architecture without pretrained embedding (2conv1d layer)
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,trainable = True,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [49]:
# basic cnn architecture without pretrained embedding (2conv1d layer)
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,trainable = False,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [28]:
# basic cnn architecture without pretrained embedding (3conv1d layer)
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [62]:
# basic cnn architecture without pretrained embedding (3conv1d layer) CNN Static
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,trainable = False,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [59]:
# basic cnn architecture with pretraining
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,weights=[embedding_matrix],trainable=True,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [124]:
# basic cnn architecture with pretraining CNN Static
model = Sequential()
model.add(Embedding(vocabulary_size,embedding_dim,weights=[embedding_matrix],trainable=False,input_length = max(max_length_train,max_length_validation)))
model.add(Conv1D(100,2, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [70]:
# more complex cnn architecture from kaggle notebook
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

pretrained = 1
if pretrained:
  embedding_layer = Embedding(vocabulary_size,
                            embedding_dim,
                            weights=[embedding_matrix],
                            trainable=True)
else:
  embedding_layer = Embedding(vocabulary_size,
                            embedding_dim)

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], embedding_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
#print(merged_tensor.shape)
flatten = Flatten()(merged_tensor)
#print(flatten.shape)
reshape = Reshape((3*num_filters,))(flatten)

dropout = Dropout(drop)(flatten)
output = Dense(units=5, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)
#inputs = Input(shape=(sequence_length,))
#embedding = embedding_layer(inputs)
#reshape = Reshape((sequence_length,embedding_dim,1))(embedding)
#output = Dense(units=5, activation='softmax')

# this creates a model that includes
#model = Model(inputs, output)

# Reference:
#https://www.kaggle.com/marijakekic/cnn-in-keras-with-pretrained-word2vec-weights

In [125]:
#optimizer = Adam(lr=0.0001,decay=0.0, amsgrad=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 7513, 100)         16652600  
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 7513, 100)         20100     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 3756, 100)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 375600)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               192307712 
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 2565      
Total params: 208,982,977
Trainable params: 192,330,377
Non-trainable params: 16,652,600
_______________________________

In [126]:
sequences_test=tokenizer.texts_to_sequences(test_data.text)
X_test = pad_sequences(sequences_test,maxlen=max(max_length_train,max_length_validation))
y_test = test_data['label']
y_test = label_encoder.fit_transform(y_test)

In [40]:
# train with basic cnn layer without pretrained embedding (1 convd layer) CNN static
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 47s 231ms/step - loss: 2.2244 - acc: 0.4456 - val_loss: 1.0013 - val_acc: 0.6081
Epoch 2/4
200/200 [==============================] - 46s 231ms/step - loss: 0.7637 - acc: 0.7211 - val_loss: 0.7519 - val_acc: 0.7125
Epoch 3/4
200/200 [==============================] - 46s 231ms/step - loss: 0.3712 - acc: 0.8918 - val_loss: 0.7140 - val_acc: 0.7231
Epoch 4/4
50/50 [==============================] - 7s 137ms/step - loss: 0.6931 - acc: 0.7556
Validation Accuracy: 75.56%
63/63 [==============================] - 9s 135ms/step - loss: 0.6874 - acc: 0.7555
Test Accuracy: 0.7555000185966492


In [20]:
# train with basic cnn layer without pretrained embedding (1 convd layer)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 184s 883ms/step - loss: 2.1094 - acc: 0.5265 - val_loss: 0.4512 - val_acc: 0.8369
Epoch 2/4
200/200 [==============================] - 176s 879ms/step - loss: 0.1687 - acc: 0.9481 - val_loss: 0.5092 - val_acc: 0.8281
Epoch 3/4
200/200 [==============================] - 176s 878ms/step - loss: 0.0113 - acc: 0.9979 - val_loss: 0.5544 - val_acc: 0.8369
Epoch 4/4
50/50 [==============================] - 6s 115ms/step - loss: 0.5583 - acc: 0.8450
Accuracy: 84.50%


In [26]:
# train with basic cnn layer without pretrained embedding (2 convd layer)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 175s 870ms/step - loss: 1.2571 - acc: 0.4849 - val_loss: 0.5222 - val_acc: 0.8094
Epoch 2/4
200/200 [==============================] - 173s 866ms/step - loss: 0.2610 - acc: 0.9094 - val_loss: 0.4799 - val_acc: 0.8356
Epoch 3/4
200/200 [==============================] - 173s 864ms/step - loss: 0.0227 - acc: 0.9926 - val_loss: 0.6454 - val_acc: 0.8394
Epoch 4/4
50/50 [==============================] - 6s 118ms/step - loss: 0.6730 - acc: 0.8531
Validation Accuracy: 85.31%
63/63 [==============================] - 8s 118ms/step - loss: 0.7600 - acc: 0.8285
Test Accuracy: 0.828499972820282


In [52]:
# train with basic cnn layer without pretrained embedding (2 convd layer) CNN static
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 46s 227ms/step - loss: 1.5722 - acc: 0.4114 - val_loss: 1.1091 - val_acc: 0.5069
Epoch 2/4
200/200 [==============================] - 45s 227ms/step - loss: 1.0196 - acc: 0.5567 - val_loss: 0.8301 - val_acc: 0.6562
Epoch 3/4
200/200 [==============================] - 45s 227ms/step - loss: 0.6580 - acc: 0.7353 - val_loss: 0.7121 - val_acc: 0.7306
Epoch 4/4
50/50 [==============================] - 7s 139ms/step - loss: 0.6406 - acc: 0.7681
Validation Accuracy: 76.81%
63/63 [==============================] - 9s 138ms/step - loss: 0.6381 - acc: 0.7675
Test Accuracy: 0.7674999833106995


In [31]:
# train with basic cnn layer without pretrained embedding (3 convd layer)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 175s 871ms/step - loss: 1.2733 - acc: 0.4669 - val_loss: 0.5768 - val_acc: 0.7825
Epoch 2/4
200/200 [==============================] - 173s 867ms/step - loss: 0.3444 - acc: 0.8809 - val_loss: 0.5386 - val_acc: 0.8169
Epoch 3/4
200/200 [==============================] - 173s 866ms/step - loss: 0.0555 - acc: 0.9831 - val_loss: 1.0221 - val_acc: 0.7681
Epoch 4/4
50/50 [==============================] - 7s 130ms/step - loss: 1.0593 - acc: 0.7894
Validation Accuracy: 78.94%
63/63 [==============================] - 8s 130ms/step - loss: 1.0100 - acc: 0.7915
Test Accuracy: 0.7914999723434448


In [65]:
# train with basic cnn layer without pretrained embedding (3 convd layer) CNN static
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 45s 223ms/step - loss: 1.4258 - acc: 0.3893 - val_loss: 1.1428 - val_acc: 0.4737
Epoch 2/4
200/200 [==============================] - 44s 222ms/step - loss: 1.0630 - acc: 0.5250 - val_loss: 0.7998 - val_acc: 0.6744
Epoch 3/4
200/200 [==============================] - 45s 223ms/step - loss: 0.6490 - acc: 0.7379 - val_loss: 0.6576 - val_acc: 0.7650
Epoch 4/4
50/50 [==============================] - 7s 140ms/step - loss: 0.6121 - acc: 0.7800
Validation Accuracy: 78.00%
63/63 [==============================] - 9s 140ms/step - loss: 0.6435 - acc: 0.7770
Test Accuracy: 0.7770000100135803


In [61]:
# train with basic cnn layer without pretrained embedding on large Turkish pretrained model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 175s 874ms/step - loss: 3.2832 - acc: 0.6163 - val_loss: 0.5360 - val_acc: 0.8213
Epoch 2/4
200/200 [==============================] - 175s 875ms/step - loss: 0.2240 - acc: 0.9304 - val_loss: 0.4711 - val_acc: 0.8306
Epoch 3/4
200/200 [==============================] - 169s 843ms/step - loss: 0.0447 - acc: 0.9921 - val_loss: 0.6008 - val_acc: 0.8281
Epoch 4/4
50/50 [==============================] - 6s 126ms/step - loss: 0.6682 - acc: 0.8369
Validation Accuracy: 83.69%
63/63 [==============================] - 8s 125ms/step - loss: 0.7188 - acc: 0.8315
Test Accuracy: 0.8314999938011169


In [110]:
# train with basic cnn layer without pretrained embedding on large Turkish pretrained model (CNN Static)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 45s 220ms/step - loss: 3.1740 - acc: 0.5988 - val_loss: 0.5367 - val_acc: 0.8225
Epoch 2/4
200/200 [==============================] - 44s 219ms/step - loss: 0.2394 - acc: 0.9231 - val_loss: 0.5345 - val_acc: 0.8169
Epoch 3/4
200/200 [==============================] - 44s 219ms/step - loss: 0.0727 - acc: 0.9824 - val_loss: 0.6311 - val_acc: 0.8175
Epoch 4/4
50/50 [==============================] - 6s 124ms/step - loss: 0.6783 - acc: 0.8294
Validation Accuracy: 82.94%
63/63 [==============================] - 8s 124ms/step - loss: 0.7190 - acc: 0.8265
Test Accuracy: 0.8264999985694885


In [45]:
# train with basic cnn layer without pretrained embedding on my 100k word2vec model in project 2
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 136s 678ms/step - loss: 1.5909 - acc: 0.5973 - val_loss: 0.4722 - val_acc: 0.8456
Epoch 2/4
200/200 [==============================] - 135s 673ms/step - loss: 0.1729 - acc: 0.9467 - val_loss: 0.6332 - val_acc: 0.8006
Epoch 3/4
200/200 [==============================] - 134s 672ms/step - loss: 0.0241 - acc: 0.9959 - val_loss: 0.5211 - val_acc: 0.8519
Epoch 4/4
50/50 [==============================] - 4s 90ms/step - loss: 0.5888 - acc: 0.8469
Validation Accuracy: 84.69%
63/63 [==============================] - 6s 90ms/step - loss: 0.6057 - acc: 0.8365
Test Accuracy: 0.8364999890327454


In [95]:
# train with basic cnn layer without pretrained embedding on my 100k word2vec model in project 2 (CNN Static)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 38s 189ms/step - loss: 1.9691 - acc: 0.5433 - val_loss: 0.5116 - val_acc: 0.8200
Epoch 2/4
200/200 [==============================] - 38s 189ms/step - loss: 0.2619 - acc: 0.9131 - val_loss: 0.4665 - val_acc: 0.8344
Epoch 3/4
200/200 [==============================] - 38s 189ms/step - loss: 0.0765 - acc: 0.9859 - val_loss: 0.5227 - val_acc: 0.8363
Epoch 4/4
50/50 [==============================] - 5s 104ms/step - loss: 0.5936 - acc: 0.8419
Validation Accuracy: 84.19%
63/63 [==============================] - 7s 105ms/step - loss: 0.6489 - acc: 0.8335
Test Accuracy: 0.8335000276565552


In [56]:
# train with basic cnn layer without pretrained embedding on my 5k   word2vec model in project 2
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) 
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 55s 272ms/step - loss: 1.8825 - acc: 0.5615 - val_loss: 0.5182 - val_acc: 0.8050
Epoch 2/4
200/200 [==============================] - 54s 271ms/step - loss: 0.2708 - acc: 0.9004 - val_loss: 0.4653 - val_acc: 0.8275
Epoch 3/4
200/200 [==============================] - 56s 278ms/step - loss: 0.0749 - acc: 0.9856 - val_loss: 0.5799 - val_acc: 0.8100
Epoch 4/4
50/50 [==============================] - 2s 37ms/step - loss: 0.5445 - acc: 0.8381
Validation Accuracy: 83.81%
63/63 [==============================] - 2s 38ms/step - loss: 0.6131 - acc: 0.8215
Test Accuracy: 0.8215000033378601


In [ ]:
# train with kaggle approach
#Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 64,shuffle = True) # train without pretrained model
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
100/100 [==============================] - 4201s 42s/step - loss: 1.5927 - acc: 0.4370 - val_loss: 1.0200 - val_acc: 0.7719
Epoch 2/4
100/100 [==============================] - 4216s 42s/step - loss: 0.9447 - acc: 0.7838 - val_loss: 0.8573 - val_acc: 0.7981
Epoch 3/4
100/100 [==============================] - 4296s 43s/step - loss: 0.7749 - acc: 0.8443 - val_loss: 0.7953 - val_acc: 0.8163
Epoch 4/4
50/50 [==============================] - 234s 5s/step - loss: 0.7923 - acc: 0.8131
Accuracy: 81.31%


In [ ]:
# train with kaggle approach
#Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) # train with my 5k pretrained model
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
200/200 [==============================] - 1169s 6s/step - loss: 1.4051 - acc: 0.5233 - val_loss: 1.0788 - val_acc: 0.6963
Epoch 2/4
200/200 [==============================] - 1171s 6s/step - loss: 0.9756 - acc: 0.7433 - val_loss: 0.9156 - val_acc: 0.7600
Epoch 3/4
200/200 [==============================] - 1173s 6s/step - loss: 0.8093 - acc: 0.8289 - val_loss: 0.8814 - val_acc: 0.7663
Epoch 4/4
50/50 [==============================] - 59s 1s/step - loss: 0.8937 - acc: 0.7900
Accuracy: 79.00%


In [127]:
# train with basic cnn layer without pretrained embedding on my 5k word2vec model in project 2 (CNN Static)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) # train with my 5k pretrained model
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 21s 101ms/step - loss: 1.9737 - acc: 0.5581 - val_loss: 0.5885 - val_acc: 0.7800
Epoch 2/4
200/200 [==============================] - 20s 101ms/step - loss: 0.3354 - acc: 0.8838 - val_loss: 0.6089 - val_acc: 0.7987
Epoch 3/4
200/200 [==============================] - 20s 101ms/step - loss: 0.1184 - acc: 0.9745 - val_loss: 0.6393 - val_acc: 0.7875
Epoch 4/4
50/50 [==============================] - 2s 40ms/step - loss: 0.7009 - acc: 0.7981
Validation Accuracy: 79.81%
63/63 [==============================] - 3s 41ms/step - loss: 0.6862 - acc: 0.8040
Test Accuracy: 0.8040000200271606


In [ ]:
# train with kaggle approach
#Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) # train with my 100k pretrained model
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
200/200 [==============================] - 161s 765ms/step - loss: 1.6377 - acc: 0.4203 - val_loss: 1.1997 - val_acc: 0.7588
Epoch 2/4
200/200 [==============================] - 150s 749ms/step - loss: 1.1289 - acc: 0.7983 - val_loss: 1.0773 - val_acc: 0.8062
Epoch 3/4
200/200 [==============================] - 150s 749ms/step - loss: 0.9673 - acc: 0.8617 - val_loss: 1.0433 - val_acc: 0.8044
Epoch 4/4
50/50 [==============================] - 7s 134ms/step - loss: 0.9811 - acc: 0.8219
Accuracy: 82.19%


In [73]:
# train with kaggle approach
#Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) # train with my 100k with more complex kaggle CNN architecture
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Accuracy: %.2f%%" % (scores[1]*100))x
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 181s 895ms/step - loss: 1.2300 - acc: 0.6500 - val_loss: 0.9116 - val_acc: 0.8356
Epoch 2/4
200/200 [==============================] - 178s 892ms/step - loss: 0.8598 - acc: 0.8636 - val_loss: 0.8339 - val_acc: 0.8550
Epoch 3/4
200/200 [==============================] - 179s 895ms/step - loss: 0.7172 - acc: 0.9061 - val_loss: 0.8112 - val_acc: 0.8512
Epoch 4/4
50/50 [==============================] - 10s 191ms/step - loss: 0.7885 - acc: 0.8544
Accuracy: 85.44%
63/63 [==============================] - 12s 195ms/step - loss: 0.7985 - acc: 0.8450
Test Accuracy: 0.8450000286102295


In [67]:
#Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 32,shuffle = True) # train with trmodel with more complex kaggle CNN architecture
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Validation Accuracy: %.2f%%" % (scores[1]*100))
print("Test Accuracy:", model.evaluate(X_test,y_test)[1])

Epoch 1/4
200/200 [==============================] - 221s 1s/step - loss: 1.4157 - acc: 0.6345 - val_loss: 1.0459 - val_acc: 0.8400
Epoch 2/4
200/200 [==============================] - 217s 1s/step - loss: 1.1093 - acc: 0.8122 - val_loss: 1.1032 - val_acc: 0.8169
Epoch 3/4
200/200 [==============================] - 218s 1s/step - loss: 0.9791 - acc: 0.8535 - val_loss: 0.9814 - val_acc: 0.8363
Epoch 4/4
50/50 [==============================] - 12s 240ms/step - loss: 0.9085 - acc: 0.8656
Validation Accuracy: 86.56%
63/63 [==============================] - 16s 244ms/step - loss: 0.9326 - acc: 0.8410
Test Accuracy: 0.8410000205039978


In [ ]:
# train with kaggle approach
#Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 64) # train with simpler model
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
100/100 [==============================] - 61s 605ms/step - loss: 3.2821 - acc: 0.5511 - val_loss: 0.5358 - val_acc: 0.8169
Epoch 2/4
100/100 [==============================] - 60s 604ms/step - loss: 0.3087 - acc: 0.9000 - val_loss: 0.4706 - val_acc: 0.8313
Epoch 3/4
100/100 [==============================] - 61s 606ms/step - loss: 0.0873 - acc: 0.9761 - val_loss: 0.5452 - val_acc: 0.8200
Epoch 4/4
50/50 [==============================] - 9s 171ms/step - loss: 0.6272 - acc: 0.8344
Accuracy: 83.44%


In [ ]:
# train with kaggle approach
#Fitting the data onto model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4,batch_size = 64) # train with label encoder version
# Getting score metrics from our model
scores = model.evaluate(X_val, y_val)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
100/100 [==============================] - 61s 606ms/step - loss: 3.4908 - acc: 0.5325 - val_loss: 0.5312 - val_acc: 0.8081
Epoch 2/4
100/100 [==============================] - 60s 604ms/step - loss: 0.3217 - acc: 0.8914 - val_loss: 0.4870 - val_acc: 0.8275
Epoch 3/4
100/100 [==============================] - 60s 605ms/step - loss: 0.1248 - acc: 0.9647 - val_loss: 0.5764 - val_acc: 0.8219
Epoch 4/4
50/50 [==============================] - 9s 173ms/step - loss: 0.6113 - acc: 0.8300
Accuracy: 83.00%
